In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from catboost import CatBoostRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, make_scorer
from IPython.display import FileLink
# from tqdm import tqdm_notebook 
from tqdm import tqdm
from xgboost.sklearn import XGBRegressor
from sklearn.model_selection import GridSearchCV

from sklearn.model_selection import PredefinedSplit
import warnings
from joblib import Parallel, delayed
from tsfresh.utilities.dataframe_functions import roll_time_series, make_forecasting_frame
from tsfresh.utilities.dataframe_functions import impute
warnings.filterwarnings('ignore')


In [2]:
train_file = 'training_files/result_{}.csv'
super_train = 'super_train.csv'


In [3]:
stock = 1


In [4]:
tdf = pd.read_csv(super_train, index_col='ID')

tdf

,Open_hat,Open_trend,Open_lower,Open_upper,Open_trend_lower,Open_ trend_upper,High_hat,High_trend,High_lower,High_upper,...,Open,High,Low,Close,holiday,unpredictability_score,Open_diff,Low_diff,High_diff,Close_diff
ID,,,,,,,,,,,,,,,,,,,,,
id_0,81.025967,82.386498,78.803918,83.056482,82.386498,82.386498,81.202899,82.597286,79.340480,83.157889,...,82.9961,82.7396,82.9144,82.8101,1,7,1.970133,2.002479,1.536701,1.975953
id_1,80.865875,82.437997,78.822365,82.974286,82.437997,82.437997,81.046504,82.643819,79.061205,83.051594,...,83.1312,83.1669,83.3779,82.9690,0,7,2.265325,2.466027,2.120396,2.151123
id_2,80.976781,82.489497,78.898351,83.082205,82.489497,82.489497,81.080195,82.690352,79.154399,83.025130,...,82.6622,82.7634,82.8984,82.8578,0,7,1.685419,1.803884,1.683205,1.912449
id_3,81.060195,82.540996,78.763372,83.062250,82.540996,82.540996,81.148817,82.736884,79.077222,83.003408,...,83.0279,82.7950,82.8425,82.7385,0,7,1.967705,1.628570,1.646183,1.651943
id_4,81.141042,82.695494,79.101468,83.174879,82.695494,82.695494,81.171321,82.876482,78.954744,83.127788,...,82.3761,82.0828,82.1473,81.8641,0,7,1.235058,0.733757,0.911479,0.672735
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
id_77657,133.246558,130.185331,130.398763,135.963138,128.413669,131.840591,132.715079,129.880891,129.605239,135.501505,...,NaN,NaN,NaN,NaN,0,6,0.153895,0.051313,0.258674,0.197796
id_77658,133.613717,130.157991,130.816426,136.724601,128.277605,131.948423,133.031871,129.850858,129.984751,135.854295,...,NaN,NaN,NaN,NaN,1,6,0.457144,0.291064,0.598355,0.414344
id_77659,133.402365,130.144321,130.475015,136.215723,128.193173,132.004618,132.892882,129.835841,129.924311,135.739193,...,NaN,NaN,NaN,NaN,0,6,0.152253,0.143879,0.246128,0.265842


In [5]:
cat_cols = [ 
    'holiday',
    'stock',
    'day',
     'month',
     'year',
     'dayofweek',
     'dayofyear',
     'weekofyear',
    'unpredictability_score']
excluded_cols = ['Close_hat', 'Open_hat', 'High_hat', 'Low_hat']
                 

In [6]:
def expand_df(dframe):
    dFrame = dframe.copy()
    dFrame['day'] = dFrame.Date.apply(lambda x: x.day)
    dFrame['month'] = dFrame.Date.apply(lambda x: x.month)
    dFrame['year'] = dFrame.Date.apply(lambda x: x.year)
    dFrame['dayofweek'] = dFrame.Date.apply(lambda x: x.dayofweek)
    dFrame['dayofyear'] = dFrame.Date.apply(lambda x: x.dayofyear)
    dFrame['weekofyear'] = dFrame.Date.apply(lambda x: x.weekofyear)
    dFrame['year_diff'] = dFrame.Date.apply(lambda x: x.year - 2017)
    dFrame['days_so_far_skipped'] = dFrame.Date.apply(lambda x: dFrame[dFrame.Date < x].shape[0])
    dFrame['days_so_far'] = dFrame.Date.apply(lambda x: (x - pd.Timestamp('2017-01-03')).days)
    return dFrame
#     return dFrame[['Open_hat', 'High_hat', 'Low_hat', 'Close_hat'] + ['Date'] + cat_cols + ['year_diff', 'days_so_far_skipped', 'days_so_far'] + ['Close', 'Open', 'High', 'Low'] ]

In [7]:
def get_rolling_mean(dframe, col, idx, days=30, met='mean'):
#     print(col, idx, days, met)
    if met == 'mean':
        return dframe[(dframe['days_so_far_skipped'] < dframe.loc[idx]['days_so_far_skipped'] + days) & (dframe['days_so_far_skipped'] >= dframe.loc[idx]['days_so_far_skipped'])][col].mean()
    if met == 'max':
        return dframe[(dframe['days_so_far_skipped'] < dframe.loc[idx]['days_so_far_skipped'] + days) & (dframe['days_so_far_skipped'] >= dframe.loc[idx]['days_so_far_skipped'])][col].max()
    return dframe[(dframe['days_so_far_skipped'] < dframe.loc[idx]['days_so_far_skipped'] + days) & (dframe['days_so_far_skipped'] >= dframe.loc[idx]['days_so_far_skipped'])][col].min()

def rolled_mean(dframe, timeshift=30):
    dframe['ID'] = dframe.index

    for col in excluded_cols:
        dframe[col + '_roll_mean_per_mon'] = dframe['ID'].apply(lambda x: get_rolling_mean(dframe, col, x, days=timeshift, met='mean'))
        dframe[col + '_roll_max_per_mon'] = dframe['ID'].apply(lambda x: get_rolling_mean(dframe, col, x, days=timeshift, met='max'))
        dframe[col + '_roll_min_per_mon'] = dframe['ID'].apply(lambda x:get_rolling_mean(dframe, col, x, days=timeshift, met='min'))
        dframe[col + '_roll_range_per_mon'] = dframe[col + '_roll_max_per_mon'] - dframe[col + '_roll_min_per_mon']
        dframe[col + '_roll_mean_per_d'] = dframe['ID'].apply(lambda x: get_rolling_mean(dframe, col, x, days=2, met='mean'))
        dframe[col + '_roll_max_per_d'] = dframe['ID'].apply(lambda x: get_rolling_mean(dframe, col, x, days=2, met='max'))
        dframe[col + '_roll_min_per_d'] = dframe['ID'].apply(lambda x:get_rolling_mean(dframe, col, x, days=2, met='min'))
        dframe[col + '_roll_range_per_d'] = dframe[col + '_roll_max_per_d'] - dframe[col + '_roll_min_per_d']
    
    return dframe.set_index('ID', drop=True)

In [8]:
def split_time_series(ts, ys, train_size):
    
    split_ind = int(train_size*ts.shape[0])
    train_ts, val_ts, train_ys, val_ys = ts[:split_ind], ts[split_ind:], ys[:split_ind], ys[split_ind:]
    return (train_ts, val_ts, train_ys, val_ys)

In [9]:
df = pd.read_csv(train_file.format(stock), index_col='ID', parse_dates=['Date'])
df = df.join(tdf[['Open_hat', 'High_hat', 'Low_hat', 'Close_hat']])
df = expand_df(df)

df = rolled_mean(df)

encoder = LabelEncoder()
for col in cat_cols:
    df[col] = encoder.fit_transform(df[col])

df.columns = ["".join (c if c.isalnum() else "_" for c in str(x)) for x in df.columns]

X, y = df[df['Close'].notna()].drop(columns=['Close', 'Open', 'High', 'Low'], axis=1), df[['Close', 'stock']][df['Close'].notna()]
X_test, y_test = df[df['Close'].isna()].drop(columns=['Close', 'Open', 'High', 'Low'], axis=1), df[['Close', 'stock']][df['Close'].isna()]

X_tr, X_val, y_tr, y_val = split_time_series(X, y['Close'], train_size=.8)


In [10]:
df.columns.tolist()

['stock',
 'unpredictability_score',
 'Close',
 'High',
 'Low',
 'Open',
 'holiday',
 'Date',
 'Open_upper__benford_correlation',
 'Open_hat__benford_correlation',
 'Low_hat__benford_correlation',
 'Low_trend__benford_correlation',
 'Low_trend_lower__benford_correlation',
 'Low__trend_upper__benford_correlation',
 'Close_upper__benford_correlation',
 'Open_hat__fft_coefficient__attr__abs___coeff_0',
 'Open_hat__fft_coefficient__attr__real___coeff_0',
 'Open_hat__cwt_coefficients__coeff_0__w_20__widths__2__5__10__20_',
 'Open_hat__cwt_coefficients__coeff_0__w_10__widths__2__5__10__20_',
 'Open_hat__cwt_coefficients__coeff_0__w_5__widths__2__5__10__20_',
 'Open_hat__cwt_coefficients__coeff_0__w_2__widths__2__5__10__20_',
 'Open_hat__sum_values',
 'Open_hat__quantile__q_0_8',
 'Open_hat__abs_energy',
 'Open_hat__median',
 'Open_hat__mean',
 'Open_hat__quantile__q_0_9',
 'Open_hat__minimum',
 'Open_hat__maximum',
 'Open_hat__quantile__q_0_2',
 'Open_hat__quantile__q_0_3',
 'Open_hat__quant

In [11]:
X

,stock,unpredictability_score,holiday,Date,Open_upper__benford_correlation,Open_hat__benford_correlation,Low_hat__benford_correlation,Low_trend__benford_correlation,Low_trend_lower__benford_correlation,Low__trend_upper__benford_correlation,...,High_hat_roll_min_per_d,High_hat_roll_range_per_d,Low_hat_roll_mean_per_mon,Low_hat_roll_max_per_mon,Low_hat_roll_min_per_mon,Low_hat_roll_range_per_mon,Low_hat_roll_mean_per_d,Low_hat_roll_max_per_d,Low_hat_roll_min_per_d,Low_hat_roll_range_per_d
ID,,,,,,,,,,,,,,,,,,,,,
id_754,0,0,1,2017-01-03,0.062915,0.062915,0.062915,0.062915,0.062915,0.062915,...,38.040274,0.039247,38.824593,39.048402,38.501096,0.547305,38.528666,38.556235,38.501096,0.055138
id_755,0,0,0,2017-01-04,0.062915,0.062915,0.062915,0.062915,0.062915,0.062915,...,38.079520,0.033626,38.833514,39.048402,38.556235,0.492167,38.579376,38.602518,38.556235,0.046283
id_756,0,0,0,2017-01-05,0.062915,0.062915,0.062915,0.062915,0.062915,0.062915,...,38.113146,0.051561,38.841296,39.048402,38.602518,0.445884,38.619642,38.636765,38.602518,0.034247
id_757,0,0,0,2017-01-06,0.062915,0.062915,0.062915,0.062915,0.062915,0.062915,...,38.164707,0.140384,38.847847,39.048402,38.636765,0.411637,38.708924,38.781083,38.636765,0.144318
id_758,0,0,0,2017-01-09,0.062915,0.062915,0.062915,0.062915,0.062915,0.062915,...,38.305092,0.080633,38.857223,39.048402,38.694696,0.353706,38.812747,38.844411,38.781083,0.063328
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
id_1462,0,0,0,2019-10-25,-0.064614,-0.064614,-0.064614,-0.064614,-0.064614,-0.064614,...,45.420615,0.082395,45.487828,45.906257,45.287613,0.618644,45.882964,45.906257,45.859671,0.046587
id_1463,0,0,0,2019-10-28,-0.064614,-0.064614,-0.064614,-0.064614,-0.064614,-0.064614,...,45.420615,0.003451,45.464898,45.859671,45.218351,0.641320,45.858177,45.859671,45.856682,0.002989
id_1464,0,0,0,2019-10-29,-0.064614,-0.064614,-0.064614,-0.064614,-0.064614,-0.064614,...,45.361671,0.062395,45.443148,45.856682,45.207164,0.649518,45.836807,45.856682,45.816932,0.039750


In [12]:
def rmse(y_true, y_pred):
    return abs(mean_squared_error(y_true, y_pred, squared=False))

In [13]:
model_store1 = [0] * 103
metrics1 = [0] * 103
# df = None

params = {'learning_rate': [.3, .35, .45, .4, .5], 'depth': [2, 3, 1, 4], 'od_wait': [15, 20, 25, 10]}

preds1 = []

def get_predictions(stock):
    df = pd.read_csv(train_file.format(stock), index_col='ID', parse_dates=['Date'])
    df = df.join(tdf[['Open_hat', 'High_hat', 'Low_hat', 'Close_hat']])
    df = expand_df(df)

    df = rolled_mean(df)

    encoder = LabelEncoder()
    for col in cat_cols:
        df[col] = encoder.fit_transform(df[col])

    df.columns = ["".join (c if c.isalnum() else "_" for c in str(x)) for x in df.columns]

    X, y = df[df['Close'].notna()].drop(columns=['Close', 'Open', 'High', 'Low'], axis=1), df[['Close']][df['Close'].notna()]
    X_test, y_test = df[df['Close'].isna()].drop(columns=['Close', 'Open', 'High', 'Low'], axis=1), df[['Close']][df['Close'].isna()]

    X_tr, X_val, y_tr, y_val = split_time_series(X, y, train_size=.8)
    train_indices = np.full((X_tr.shape[0],), -1, dtype=int)
    test_indices = np.full((X_val.shape[0],), 0, dtype=int)
    
    test_fold = np.append(train_indices, test_indices)
    
    ps = PredefinedSplit(test_fold)
    
    model = CatBoostRegressor(
        loss_function='RMSE', 
#         depth=2, 
#         learning_rate=0.4, 
        iterations=800,
        random_seed=18,
        od_type='Iter',
#         od_wait=20,
        thread_count=1, task_type="GPU",
        verbose=False
    )
    
    
#     model.fit(
#         X_tr, y_tr, use_best_model=True,
#         cat_features=cat_cols,
#         eval_set=(X_val, y_val),
#         verbose=False,
#         plot=False,
#     )
    results = model.grid_search(
        params, X, y=y, cv=ps, shuffle=False, verbose=False
        
    )
#     print(model.best_results_)
#     print(np.min(results['cv_results']['test-RMSE-mean']))
    return pd.DataFrame({'Score':np.min(results['cv_results']['test-RMSE-mean']), 'Params': str(results['params']), 'ID': X_test.index, 'Close': model.predict(X_test)})


num_cores = 2
preds1 = Parallel(n_jobs=num_cores)(delayed(get_predictions)(stock) for stock in tqdm(range(103)))
# preds1 = [get_predictions(stock) for stock in tqdm(range(1))]

100%|██████████| 103/103 [4:08:03<00:00, 144.50s/it] 


In [14]:

pd.concat(preds1)[['ID', 'Close']].to_csv('result_cat1.csv', index=False)

pd.concat(preds1).to_csv('result_catboost_analysis1.csv', index=False)



In [15]:
pd.concat(preds1)

,Score,Params,ID,Close
0,30.966007,"{'depth': 1, 'od_wait': 15, 'learning_rate': 0.3}",id_713,118.082441
1,30.966007,"{'depth': 1, 'od_wait': 15, 'learning_rate': 0.3}",id_714,118.082441
2,30.966007,"{'depth': 1, 'od_wait': 15, 'learning_rate': 0.3}",id_715,118.082441
3,30.966007,"{'depth': 1, 'od_wait': 15, 'learning_rate': 0.3}",id_716,118.139719
4,30.966007,"{'depth': 1, 'od_wait': 15, 'learning_rate': 0.3}",id_717,118.139719
...,...,...,...,...
36,2.708074,"{'depth': 1, 'od_wait': 15, 'learning_rate': 0.4}",id_77657,135.212532
37,2.708074,"{'depth': 1, 'od_wait': 15, 'learning_rate': 0.4}",id_77658,135.269638
38,2.708074,"{'depth': 1, 'od_wait': 15, 'learning_rate': 0.4}",id_77659,135.355410
39,2.708074,"{'depth': 1, 'od_wait': 15, 'learning_rate': 0.4}",id_77660,135.298305


In [16]:
FileLink('result_cat1.csv')

/home/ashutosh/code/AnalyticsVidhya_JANATAHACK/stocks/result_cat1.csv

In [17]:
FileLink('result_catboost_analysis1.csv')

/home/ashutosh/code/AnalyticsVidhya_JANATAHACK/stocks/result_catboost_analysis1.csv

In [18]:

!cat result_latest.csv

cat: result_latest.csv: No such file or directory


In [19]:
!cat result_catboost_analysis.csv

cat: result_catboost_analysis.csv: No such file or directory


In [20]:
pd.concat(preds1)['Params'].unique()

array(["{'depth': 1, 'od_wait': 15, 'learning_rate': 0.3}",
       "{'depth': 2, 'od_wait': 15, 'learning_rate': 0.5}",
       "{'depth': 1, 'od_wait': 20, 'learning_rate': 0.3}",
       "{'depth': 1, 'od_wait': 15, 'learning_rate': 0.45}",
       "{'depth': 1, 'od_wait': 25, 'learning_rate': 0.3}",
       "{'depth': 2, 'od_wait': 15, 'learning_rate': 0.35}",
       "{'depth': 2, 'od_wait': 15, 'learning_rate': 0.4}",
       "{'depth': 1, 'od_wait': 20, 'learning_rate': 0.4}",
       "{'depth': 1, 'od_wait': 20, 'learning_rate': 0.5}",
       "{'depth': 1, 'od_wait': 25, 'learning_rate': 0.45}",
       "{'depth': 1, 'od_wait': 25, 'learning_rate': 0.5}",
       "{'depth': 1, 'od_wait': 15, 'learning_rate': 0.5}",
       "{'depth': 3, 'od_wait': 15, 'learning_rate': 0.4}",
       "{'depth': 1, 'od_wait': 15, 'learning_rate': 0.35}",
       "{'depth': 2, 'od_wait': 15, 'learning_rate': 0.3}",
       "{'depth': 4, 'od_wait': 15, 'learning_rate': 0.35}",
       "{'depth': 3, 'od_wait': 20,

In [21]:
pd.concat(preds1)['Score'].unique()

array([30.96600677,  1.9353204 , 94.15035958,  1.12494499,  2.99606214,
        5.3498874 , 30.4605941 ,  2.79241367,  0.56159636, 11.59841215,
        0.829241  , 16.25588079,  6.51682092,  5.05890948, 16.0869221 ,
       15.64425911, 17.93151489,  0.77686265,  5.41425332,  3.42521284,
        1.44505711,  5.88235166,  1.23412601, 13.23776587,  3.44695735,
        2.15903408,  4.18750958,  2.02677442,  0.76786057, 26.1286395 ,
        0.63812124,  1.81215816,  4.55161939,  1.52134699, 16.25496327,
        1.27111802,  9.69704301, 47.99074339,  0.87143912,  1.26037962,
        8.96026285,  4.40511593,  1.22512581,  0.70011874,  1.46060872,
        4.43192163,  3.56318404,  3.45663258,  1.86782521,  0.82431785,
       36.87770376,  1.72425612,  9.87746854,  0.94137998,  2.08575149,
        9.39707846,  1.01205519,  2.00621539,  2.64939219,  2.77344712,
       23.22099947,  1.23142068,  1.54826117,  8.46817825,  4.12841521,
        2.83402704,  6.56481707, 36.91273256,  1.43570565,  2.24